### BasicRNNCell
#### TensorFlow 0.9 implementation based on hunkim's tutorial
https://hunkim.github.io/ml/

https://www.youtube.com/watch?v=A8wJYfDUYCk&feature=youtu.be

In [1]:
import tensorflow as tf
import numpy as np

# 데이터 생성

In [ ]:
char_rdic = ['h', 'e', 'l', 'o'] # id -> char
char_dic = {w : i for i, w in enumerate(char_rdic)} # char -> id
ground_truth = [char_dic[c] for c in 'hello']
print(ground_truth)

# x값 생성

In [ ]:
x_data = tf.one_hot(ground_truth[:-1], 4, 1.0, 0.0, -1)

# rnn_size , batch_size 설정

rnn_size 는 아웃풋의 갯수  
batch_size는 batch 사이즈

In [ ]:
rnn_size = len(char_dic) # 4
batch_size = 1

# 모델생성과 첫번째 상태 초기화

https://www.quora.com/Why-using-sigmoid-and-tanh-as-the-activation-functions-in-LSTM-or-RNN-is-not-problematic-but-this-is-not-the-case-in-other-neural-nets

In [ ]:
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size,
                                       input_size = None, # deprecated at tensorflow 0.9
                                       #activation = tanh,
                                       )

initial_state = rnn_cell.zero_state(batch_size, tf.float32)

# x값을 split

x_split output 

```python
[[1,0,0,0]] # h  
[[0,1,0,0]] # e  
[[0,0,1,0]] # l  
[[0,0,0,1]] # l  
```

In [ ]:
x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split

outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)
print outputs, '\n'
print state

# OUTPUT을 RESHAPE


['h', 'e', 'l', 'o']  
logits =  
[[h일확률,e일확률,l일확률,o일확률],   첫번째 아웃풋  
        [h일확률,e일확률,l일확률,o일확률],   두번째 아웃풋  
        [h일확률,e일확률,l일확률,o일확률],  세번째 아웃풋  
        [h일확률,e일확률,l일확률,o일확률]]       네번째 아웃풋  
        
왜 concat 하지? print outputs, '\n' 이결과 [1,4,4] 의 형태라서!!   
If values[i].shape = [D0, D1, ... Dconcat_dim(i), ...Dn], the concatenated result has shape
 [D0, D1, ... Rconcat_dim, ...Dn]

In [ ]:
#tf.concat = Concatenates tensors along one dimension.

logits = tf.reshape(tf.concat(1, outputs),  # shape = 1 x 16
                    [-1, rnn_size])  # shape = 4 x 4

#y data

targets = tf.reshape(ground_truth[1:], [-1]) # [[e],[l],[l],[o]]

# weight 
weights = tf.ones([len(char_dic) * batch_size])
 """Most basic RNN: output = new_state = activation(W * input + U * state + B)."""

# cost, loss function 정의

In [ ]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [ ]:
# Launch the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        print(result, [char_rdic[t] for t in result])